In [1]:
import pandas as pd 
import numpy as np
import torch
from torch.utils.data import Dataset
from torch import nn

In [19]:
t1 = torch.Tensor([[1,4], [3,4]])
t2 = torch.Tensor([[2,2], [3,3]])

print(t1)
print(t2)


tensor([[1., 4.],
        [3., 4.]])
tensor([[2., 2.],
        [3., 3.]])


In [2]:
pd.read_pickle("../../state/new_physics/data/raw/signal/dc9_-0.01_1_re.pkl")

__experiment__  __run__  __event__  __production__  __candidate__  \
det 0                1003        0          1               0              0   
    1                1003        0          2               0              0   
    2                1003        0          4               0              0   
    3                1003        0          4               0              1   
    4                1003        0          7               0              0   
...                   ...      ...        ...             ...            ...   
gen 23995            1003        0      23996               0              0   
    23996            1003        0      23997               0              0   
    23997            1003        0      23998               0              0   
    23998            1003        0      23999               0              0   
    23999            1003        0      24000               0              0   

           __ncandidates__  __weight__       Mbc    deltaE         M  ...  \
det 0                    1         1.0  5.276268  0.013677  5.289981  ...   
    1                    1         1.0  5.278889 -0.010221  5.268647  ...   
    2                    2         1.0  5.277414 -0.004075  5.273330  ...   
    3                    2         1.0  5.278964  0.002147  5.281116  ...   
    4                    1         1.0  5.279648  0.002362  5.282014  ...   
...                    ...         ...       ...       ...       ...  ...   
gen 23995                1         1.0  5.285594 -0.005939  5.279650  ...   
    23996                1         1.0  5.277320  0.002325  5.279650  ...   
    23997                1         1.0  5.281403 -0.004129  5.277267  ...   
    23998                1         1.0  5.282717 -0.003063  5.279650  ...   
    23999                1         1.0  5.278435  0.001213  5.279650  ...   

           costheta_mu  costheta_mu_mc  costheta_K  costheta_K_mc    coschi  \
det 0         0.979380        0.979081   -0.287127      -0.275016 -0.958228   
    1         0.786068        0.788733    0.124343       0.113048 -0.950656   
    2        -0.273616       -0.275305    0.918631       0.916199  0.047295   
    3        -0.199028       -0.648885    0.717579      -0.349258 -0.276820   
    4         0.899911        0.899777    0.632428       0.642848  0.021435   
...                ...             ...         ...            ...       ...   
gen 23995    -0.397837       -0.397837   -0.928857      -0.928614 -0.931403   
    23996     0.989903        0.989903    0.101298       0.103062  0.869848   
    23997     0.717159        0.716988   -0.657532      -0.679629  0.861440   
    23998    -0.933475       -0.933475   -0.835899      -0.833507 -0.839943   
    23999     0.183296        0.183296    0.630294       0.658259 -0.984196   

           coschi_mc       chi    chi_mc  invM_K_pi_shifted  \
det 0      -0.958471  3.431647  3.430796          -0.010902   
    1      -0.950541  2.826140  2.825770           0.008385   
    2       0.044790  1.523484  1.525991          -0.016957   
    3       0.385589  4.431906  1.174950          -0.072330   
    4       0.026246  1.549360  1.544547          -0.029159   
...              ...       ...       ...                ...   
gen 23995  -0.931814  3.514139  3.513008           0.000914   
    23996   0.869774  0.515903  0.516053           0.000679   
    23997   0.861264  0.532698  0.533044           0.076015   
    23998  -0.840626  3.715212  3.713951          -0.009373   
    23999  -0.987814  2.963569  2.985316          -0.079527   

           invM_K_pi_shifted_mc  
det 0                 -0.011795  
    1                  0.010149  
    2                 -0.016767  
    3                 -0.071825  
    4                 -0.030136  
...                         ...  
gen 23995              0.000914  
    23996              0.000679  
    23997              0.076015  
    23998             -0.009373  
    23999             -0.079527  

[32887 row

In [21]:
def weird_loss(yhat, y):
    (yhat - y).mean(dim=)


l = nn.MSELoss(reduction="none")
l(t1, t2)

tensor(1.5000)

In [34]:
a = pd.DataFrame({"a":[0,1], "b":[2,3]})
a

,a,b
0,0,2
1,1,3


In [35]:
a["a"]= a["a"]*2

In [36]:
a

,a,b
0,0,2
1,2,3
